### Set up

In [7]:
import pandas as pd
import numpy as np
import altair as alt
import glob
import os
import numpy as np
from scipy import fftpack
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import FastICA
from scipy.signal import butter, filtfilt

alt.data_transformers.disable_max_rows()
fs = 22050

### Load data

In [8]:
# Which fish?
subjid = 'hydrolagusColliei_8'

loaded = np.load(f'{subjid}_data.npz', allow_pickle=True)
data = loaded['data']
freq_amp_table = loaded['freq_amp_table']
latency = loaded['latency'].item()
channel_keys = loaded['channel_keys'].tolist()

data

FileNotFoundError: [Errno 2] No such file or directory: 'hydrolagusColliei_8_data.npz'

In [9]:
data.item()[100,115]

NameError: name 'data' is not defined

### What stimulus frequencies and amplitudes were tested for this fish?

In [ ]:
freq_amp_table_df = pd.DataFrame(freq_amp_table).sort_values(by= [0,1],ascending=True)
print(list(data.item().keys()))
freq_amp_table_df

### Get data for responses to single stimulus type (frequency,amplitude)

In [ ]:
# Select which stimulus frequency and amplitude to analyze
myfreq = 100
myamp = 115

specific_key = (np.float64(myfreq), np.float64(myamp))
current_cond = data.item()[specific_key]
current_cond

### Wavelet or other signal processing technique here

In [ ]:
# 

### Reshape data for PCA/ICA

In [ ]:
# (trials x samples) x 4 channels

# Get data from channel 1
X_all_periods = current_cond['ch1'].reshape(1, -1).T
selected_keys = channel_keys[1:5]

# Then get the rest of the channels
for channel in selected_keys:
    tmp = current_cond[channel].reshape(1, -1).T
    X_all_periods = np.hstack([X_all_periods, tmp])

X_all_periods

### PCA

In [ ]:
pca = PCA(n_components=len(channel_keys))
X_pca = pca.fit_transform(X_all_periods)

print('All periods')
print("Explained variance ratio:", pca.explained_variance_ratio_)
print("Component loadings shape:", pca.components_.shape)

### Scree plot

In [ ]:
PC_values = np.arange(pca.n_components_) + 1
plt.plot(PC_values, pca.explained_variance_ratio_, 'o-', linewidth=2)
plt.title(f'All periods Scree Plot ({myfreq} Hz {myamp} dB)')
plt.xlabel('Principal Component')
plt.ylabel('Variance Explained')
plt.savefig(f'{subjid}_{myfreq}Hz_{myamp}dB_chart11.png', dpi=300, bbox_inches='tight')
plt.show()

### Loading Plot

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(10, 8))
axs = axs.ravel()  # Flatten the 2x2 array of axes for easy indexing

for i, comp in enumerate(pca.components_):
    axs[i].bar(range(1, 5), comp)
    axs[i].set_title(f"PCA Component {i+1}")
    axs[i].set_xlabel("Channel")
    axs[i].set_ylabel("Loading")

fig.suptitle(f'All Periods Loading Plot ({myfreq} Hz {myamp} dB)')
plt.savefig(f'{subjid}_{myfreq}Hz_{myamp}dB_chart12.png', dpi=300, bbox_inches='tight')
plt.show()

### ICA

#### Apply Butterworth filter

In [ ]:
def bandpass(data, low, high, fs, order=4):
    ny = 0.5 * fs
    b, a = butter(order, [low/ny, high/ny], btype='band')
    return filtfilt(b, a, data, axis=0)

# low, high = (myfreq*2-myfreq), (myfreq+myfreq*2)
low, high = 150, 250
# X_all_periods: shape (n_samples, n_channels)
X_filtered = bandpass(X_all_periods, low, high, fs)

#### Apply ICA

In [ ]:
# Preprocessing: Standardize the data
scaler = StandardScaler()

# non_filtered version
# X_all_scaled = scaler.fit_transform(X_all_periods)

# filtered version
X_all_scaled = scaler.fit_transform(X_filtered)

# Improved FastICA configuration
ica = FastICA(
    n_components=len(channel_keys), 
    random_state=42,  # More robust random seed
    max_iter=500,     # Increased iterations
    tol=1e-4,         # Tightened tolerance
    whiten='unit-variance'  # Corrected whiten parameter
)

# Fit and transform
S = ica.fit_transform(X_all_scaled)
A = ica.mixing_

# Convergence check
print(f"ICA Convergence: {ica.n_iter_}")

### Plot IC Waveforms

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(8, 6), sharey=True)

for i, ax in enumerate(axes):
    ax.plot(S[:, i])
    ax.set_title(f'Independent Component {i+1}')

fig.suptitle(f'All period IC Waveforms ({myfreq} Hz {myamp} dB)')
plt.tight_layout()
plt.savefig(f'{subjid}_{myfreq}Hz_{myamp}dB_chart13.png', dpi=300, bbox_inches='tight')
plt.show()


### Plot Mixing results

In [ ]:
# Mixing matrix table
mix_df = pd.DataFrame(ica.mixing_, index=[f'ch{i+1}' for i in range(4)],
                      columns=[f'IC{i+1}' for i in range(4)])

# Reshape the mixing matrix DataFrame to long format
mix_df_long = mix_df.reset_index().melt(
    id_vars='index', 
    var_name='IC', 
    value_name='Mixing Value'
)
mix_df_long = mix_df_long.rename(columns={'index': 'Channel'})

# Create Altair chart
chart = alt.Chart(mix_df_long).mark_line(point=True).encode(
    x=alt.X('Channel:N', title='Channel Number'),
    y=alt.Y('Mixing Value:Q', title='IC Mixing Value'),
    color=alt.Color('IC:N', title='IC Number'),
    tooltip=['Channel', 'IC', 'Mixing Value']
).properties(
    title='IC Mixing Prestim',
    width=600,
    height=400
)

chart

### Calculate FFT of each IC

In [ ]:
# Calculate FFT and power spectrum for each component
n_samples = len(S)
sampling_rate = 22050

# Get positive frequencies only
freqs = np.fft.rfftfreq(n_samples, d=1/sampling_rate)

# Calculate power spectrum for each component
power_spectra = []
for i in range(S.shape[1]):  # iterate through each component
    fft_vals = np.fft.rfft(S[:, i])
    power = np.abs(fft_vals) ** 2
    power_spectra.append(power)

# Convert to array for easier handling
power_spectra = np.array(power_spectra)  # shape: (4, n_freqs)


### Plot FFT results

In [ ]:
# Create subplots - 2x2 grid for 4 components
fig, axes = plt.subplots(2, 2, figsize=(9, 6),sharey=True)
axes = axes.flatten()

for i in range(power_spectra.shape[0]):
    axes[i].plot(freqs, power_spectra[i])
    axes[i].set_xlabel('Frequency (Hz)')
    axes[i].set_ylabel('Power')
    axes[i].set_title(f'IC {i+1}')
    axes[i].set_xlim(0, 500)
    axes[i].grid(True, alpha=0.3)

fig.suptitle(f'All periods IC FFTs ({myfreq} Hz {myamp} dB)')
plt.tight_layout()
plt.savefig(f'{subjid}_{myfreq}Hz_{myamp}dB_chart14.png', dpi=300, bbox_inches='tight')
plt.show()

### Save variables (this should be changed...)

In [ ]:
S_all = S
A_all = A
S_all

### Remove ICs to denoise

In [ ]:
# Select which ICs to remove (Depends on which)

S_modified = S_all.copy()
S_modified[:, 0] = 0  # Remove IC 1
# S_modified[:, 1] = 0  # Remove IC 2
S_modified[:, 2] = 0  # Remove IC 3
S_modified[:, 3] = 0  # Remove IC 4

# Reconstruct the signals
X_reconstructed = S_modified @ A.T

# Extract each channel and reshape to original dimensions
n_trials = current_cond['ch1'].shape[0]
n_samples_per_trial = current_cond['ch1'].shape[1]

ch1_denoised = X_reconstructed[:, 0].reshape(n_trials, n_samples_per_trial)
ch2_denoised = X_reconstructed[:, 1].reshape(n_trials, n_samples_per_trial)
ch3_denoised = X_reconstructed[:, 2].reshape(n_trials, n_samples_per_trial)
ch4_denoised = X_reconstructed[:, 3].reshape(n_trials, n_samples_per_trial)

# Create denoised dataset with same structure
denoised_data = {
    'filename': current_cond['filename'],
    'decision': current_cond['decision'],
    'period_len': current_cond['period_len'],
    'ch1': ch1_denoised,
    'ch2': ch2_denoised,
    'ch3': ch3_denoised,
    'ch4': ch4_denoised
}

### Create dicts to compare original, cleaned, and IC component data

In [ ]:
ICA_dict={}
for channel in channel_keys:
    cur_clean = denoised_data[channel]
    
    clean_mean = np.mean(cur_clean,axis=0)
    clean_std = np.std(cur_clean,axis=0)

    ICA_dict[f"{channel}_mean"] =  clean_mean
    ICA_dict[f"{channel}_std"] = clean_std

orig_dict={}
for channel in channel_keys:
    orig = current_cond[channel]
    orig_mean = np.mean(orig,axis=0)
    orig_std = np.std(orig,axis=0)
    orig_dict[f"{channel}_mean"] =  orig_mean
    orig_dict[f"{channel}_std"] = orig_std
orig_dict

load_dict={}
for ii, channel in enumerate(channel_keys):
# separate by trial again
    clean_channel = S_all[:, ii].reshape(n_trials, n_samples_per_trial)
    load_mean = np.mean(clean_channel,axis=0)
    load_std = np.std(clean_channel,axis=0)
    load_dict[f"{channel}_mean"] =  load_mean
    load_dict[f"{channel}_std"] = load_std


### Plot comparison waveforms

In [ ]:
fig, axes = plt.subplots(4, 3, figsize=(15, 7), sharex=True, sharey=True)
channels = ['ch1', 'ch2', 'ch3', 'ch4']
x = np.arange(len(ICA_dict['ch1_mean']))

for row, channel in enumerate(channels):
    # Extract data for this channel
    mean_data_clean = ICA_dict[f'{channel}_mean']
    std_data_clean = ICA_dict[f'{channel}_std']
    
    mean_data_orig = orig_dict[f'{channel}_mean']
    std_data_orig = orig_dict[f'{channel}_std']
    
    mean_data_load = load_dict[f'{channel}_mean']
    std_data_load = load_dict[f'{channel}_std']
    
    # Plot clean data (left column)
    axes[row, 0].plot(x, mean_data_clean)
    axes[row, 0].fill_between(x, 
                               mean_data_clean - std_data_clean, 
                               mean_data_clean + std_data_clean, 
                               alpha=0.3)
    axes[row, 0].set_ylabel(f'{channel}')
    
    
    # Plot original data (right column)
    axes[row, 1].plot(x, mean_data_orig,)
    axes[row, 1].fill_between(x, 
                               mean_data_orig - std_data_orig, 
                               mean_data_orig + std_data_orig, 
                               alpha=0.3)
    
    
    # Plot original data (right column)
    axes[row, 2].plot(x, mean_data_load)
    axes[row, 2].fill_between(x, 
                            mean_data_load - std_data_load, 
                            mean_data_load + std_data_load, 
                            alpha=0.3)
    

# Add column titles
axes[0, 0].set_title('Clean Data')
axes[0, 1].set_title('Original Data')
axes[0, 2].set_title('Independent Components')

# Add x-label to bottom row
axes[3, 0].set_xlabel('Sample')
axes[3, 1].set_xlabel('Sample')
axes[3, 2].set_xlabel('Sample')

fig.suptitle(f'Denoised Signals Waveforms ({myfreq} Hz {myamp} dB)')
plt.savefig(f'{subjid}_{myfreq}Hz_{myamp}dB_chart9.png', dpi=300, bbox_inches='tight')
plt.show()

### Calculate Comparison FFTs

In [ ]:
# Calculate FFT and power spectrum for each component
n_samples = len(ICA_dict['ch1_mean'])
# Get positive frequencies only
freqs = np.fft.rfftfreq(n_samples, d=1/sampling_rate)
# Calculate power spectrum for each component
# cleaned data
# mean
clean_spectra_mean = []
clean_spectra_std = []
for channel in channel_keys:  # iterate through each component
    fft_vals = np.fft.rfft(ICA_dict[f'{channel}_mean'])
    power = np.abs(fft_vals) ** 2
    clean_spectra_mean.append(power)
   
# std
for channel in channel_keys:  # iterate through each component
    fft_vals = np.fft.rfft(ICA_dict[f'{channel}_std'])
    power = np.abs(fft_vals) ** 2
    clean_spectra_std.append(power)
# original data
# mean
orig_spectra_mean = []
orig_spectra_std = []
for channel in channel_keys:  # iterate through each component
    fft_vals = np.fft.rfft(orig_dict[f'{channel}_mean'])
    power = np.abs(fft_vals) ** 2
    orig_spectra_mean.append(power)
   
# std
for channel in channel_keys:  # iterate through each component
    fft_vals = np.fft.rfft(orig_dict[f'{channel}_std'])
    power = np.abs(fft_vals) ** 2
    orig_spectra_std.append(power)
# loading data
# mean
load_spectra_mean = []
load_spectra_std = []
for channel in channel_keys:  # iterate through each component
    fft_vals = np.fft.rfft(load_dict[f'{channel}_mean'])
    power = np.abs(fft_vals) ** 2
    load_spectra_mean.append(power)
   
# std
for channel in channel_keys:  # iterate through each component
    fft_vals = np.fft.rfft(load_dict[f'{channel}_std'])
    power = np.abs(fft_vals) ** 2
    load_spectra_std.append(power)

# Convert to array for easier handling
# cleaned data
# mean
clean_spectra_mean = np.array(clean_spectra_mean)
clean_spectra_std = np.array(clean_spectra_std)
# original data
orig_spectra_mean = np.array(orig_spectra_mean)
orig_spectra_std = np.array(orig_spectra_std)
# loading data
load_spectra_mean = np.array(load_spectra_mean)
load_spectra_std = np.array(load_spectra_std)  # shape: (4, n_freqs)

### Plot comparison FFTs

In [ ]:
fig, axes = plt.subplots(4, 3, figsize=(10, 7), sharex=True, sharey=True)
channels = ['ch1', 'ch2', 'ch3', 'ch4']
x = np.arange(len(ICA_dict['ch1_mean']))

for row, channel in enumerate(channels):
    # Extract data for this channel
    mean_data_clean = clean_spectra_mean[row,:]
    std_data_clean = clean_spectra_std[row,:]
    
    mean_data_orig = orig_spectra_mean[row,:]
    std_data_orig = orig_spectra_std[row,:]
    
    mean_data_load = load_spectra_mean[row,:]
    std_data_load = load_spectra_std[row,:]
    
    # Plot clean data (left column)
    axes[row, 0].plot(freqs, mean_data_clean)
    axes[row, 0].set_ylabel(f'{channel}')

    # Plot original data (right column)
    axes[row, 1].plot(freqs, mean_data_orig,)

    # Plot original data (right column)
    axes[row, 2].plot(freqs, mean_data_load)
    
    
    # Add these lines inside your plotting loop
    axes[row, 0].set_xlim(0, 500)
    axes[row, 1].set_xlim(0, 500)
    axes[row, 2].set_xlim(0, 500)
    
    # # Add these lines inside your plotting loop
    # axes[row, 0].set_ylim(0, 100000)
    # axes[row, 1].set_ylim(0, 100000)

    




# Add column titles
axes[0, 0].set_title('Clean Data')
axes[0, 1].set_title('Original Data')
axes[0, 2].set_title('Independent Components')

# Add x-label to bottom row
axes[3, 0].set_xlabel('Frequency')
axes[3, 1].set_xlabel('Frequency')
axes[3, 2].set_xlabel('Frequency')


fig.suptitle(f'Denoised Signals FFTs ({myfreq} Hz {myamp} dB)')
plt.savefig(f'{subjid}_{myfreq}Hz_{myamp}dB_chart10.png', dpi=300, bbox_inches='tight')
plt.show()